# Семинар 5    
## Собираем поисковик 

![](https://bilimfili.com/wp-content/uploads/2017/06/bir-urune-emek-vermek-o-urune-olan-deger-algimizi-degistirir-mi-bilimfilicom.jpg) 


Мы уже все знаем, для того чтобы сделать поисковик. Осталось соединить все части вместе.    
Итак, для поисковика нам понадобятся:         
**1. База документов **
> в первом дз - корпус Друзей    
в сегодняшнем дз - корпус юридических вопросов-ответов    
в итоговом проекте - корпус Авито   

**2. Функция индексации**                 
Что делает: собирает информацию о корпусе, по которуму будет происходить поиск      
Своя для каждого поискового метода:       
> A. для обратного индекса она создает обратный индекс (чудо) и сохраняет статистики корпуса, необходимые для Okapi BM25 (средняя длина документа в коллекции, количество доков ... )             
> B. для поиска через word2vec эта функция создает вектор для каждого документа в коллекции путем, например, усреднения всех векторов коллекции       
> C. для поиска через doc2vec эта функция создает вектор для каждого документа               

   Не забывайте сохранить все, что насчитает эта функция. Если это будет происходить налету во время поиска, понятно, что он будет работать сто лет     
   
**3. Функция поиска**     
Можно разделить на две части:
1. функция вычисления близости между запросом и документом    
> 1. для индекса это Okapi BM25
> 2. для w2v и d2v это обычная косинусная близость между векторами          
2. ранжирование (или просто сортировка)


Время все это реализовать.

# Индексация
## Word2Vec
### Задание 1
Загрузите любую понравившуюся вам word2vec модель

In [2]:
#from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec, KeyedVectors
import re
import os
from tqdm import tqdm_notebook as tqdm

from gensim.models.doc2vec import Doc2Vec, TaggedDocument, TaggedLineDocument

import numpy as np
import json

import operator

In [11]:
def preprocessing(pars):
    for i in range(0, len(pars)):
        pars[i] = pars[i].replace('.', ' ')
        pars[i] = pars[i].replace(',', ' ')
        pars[i] = pars[i].replace(':', ' ')
        pars[i] = pars[i].replace(';', ' ')
        pars[i] = pars[i].replace('-', ' ')
        pars[i] = pars[i].replace(')', ' ')
        pars[i] = pars[i].replace('(', ' ')
        pars[i] = pars[i].replace('\\', ' ')
        pars[i] = pars[i].replace('/', ' ')
        pars[i] = pars[i].replace('"', ' ')
        pars[i] = pars[i].replace('!', ' ')
        pars[i] = pars[i].replace('?', ' ')
        pars[i] = pars[i].replace('\n', ' ')
        pars[i] = re.sub('[0-9]','', pars[i])
        pars[i] = pars[i].replace('   ', ' ')
        pars[i] = pars[i].replace('  ', ' ')
        pars[i] = pars[i].strip(' ')
    return pars

In [3]:
model_path = r"C:\Users\1\Downloads\all.norm-sz100-w10-cb0-it1-min100.w2v"
model = KeyedVectors.load_word2vec_format(model_path, binary=True, unicode_errors='ignore')

### Задание 2 
Напишите функцию индексации для поиска через word2vec. Она должна для каждого документа из корпуса строить вектор.   
Все вектора надо сохранить, по формату советую json. При сохранении не забывайте, что вам надо сохранить не только  вектор, но и опознователь текста, которому он принадлежит. 
Для поисковика это может быть url страницы, для поиска по текстовому корпусу сам текст.

> В качестве документа для word2vec берите **параграфы** исходного текста, а не весь текст целиком. Так вектора будут более осмысленными. В противном случае можно получить один очень общий вектор, релевантый совершенно разным запросам.

In [9]:
path = 'C:\\Users\\1\\Downloads\\Telegram Desktop\\articles\\article'

In [16]:
def make_pars(path):
    file = open(path, 'r', encoding = 'UTF-8')
    f = file.read()
    file.close()
    f = f+'. '
    f = f.lower()
    pars = re.findall('.*?[^А-Я|г]\. ', f)
    pars = preprocessing(pars) 
    return pars

In [112]:
def par_t_v(par):
    vecs = np.zeros(100)
    par = list(filter(None, par.split(' ')))
    for word in par:
        try:
            vec = model.get_vector(word)
        except Exception as ex:
            print(ex)
            continue
        if vecs.size is 0:
            vecs = vec
        else:
            vecs = vecs+vec
    vec = vecs/len(par)
    return vec

In [111]:
def write_json(pars_and_vecs, global_num):
    p = r'C:\Users\1\Downloads\Telegram Desktop\articles\json\all_json_w2v_' + str(global_num) +'.txt'
    with open(p, 'w') as f:
        json.dump(pars_and_vecs, f)
    return

In [110]:
def get_w2v_vectors(path):
    pars = make_pars(path)
    pars_and_vecs = []
    for i in range(0, len(pars)):
        vec_par = par_t_v(pars[i])
        pars_and_vecs.append(vec_par.tolist())
    return pars_and_vecs

In [113]:
global_dict = {}
global_num = 0
path = r"C:\Users\1\Downloads\Telegram Desktop\articles\article"
for idx in tqdm(range(1, 2001)):
    path_file=path+"\\"+str(idx)+'.txt'
    if os.path.isfile(path_file):
        global_dict[str(idx)] = get_w2v_vectors(path_file)
        
    if idx%100 is 0:
        global_num+=1
        write_json(global_dict, global_num)
        global_dict = {} 

A Jupyter Widget

"word 'хорошавина' not in vocabulary"
"word 'дробановой' not in vocabulary"
"word 'хорошавина' not in vocabulary"
"word 'чебоксарская' not in vocabulary"
"word 'хорошавин' not in vocabulary"
"word 'дробанова' not in vocabulary"
"word 'хорошавин' not in vocabulary"
"word 'хорошавиным' not in vocabulary"
"word 'хорошавина' not in vocabulary"
"word 'дробановой' not in vocabulary"
"word 'хорошавина' not in vocabulary"
"word 'суброгации' not in vocabulary"
"word 'обжаловалось' not in vocabulary"


C:\Users\1\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in true_divide
  


"word 'татьями' not in vocabulary"
"word 'весеневой' not in vocabulary"
"word 'весенева' not in vocabulary"
"word 'криушинский' not in vocabulary"
"word 'судостроительно' not in vocabulary"
"word 'криушинский' not in vocabulary"
"word 'судостроительно' not in vocabulary"
"word 'тесар' not in vocabulary"
"word 'тесар' not in vocabulary"
"word 'ызыл' not in vocabulary"
"word 'весенева' not in vocabulary"
"word 'химстрой' not in vocabulary"
"word 'транспорная' not in vocabulary"
"word 'химстрой' not in vocabulary"
"word 'транспорная' not in vocabulary"
"word 'весенева' not in vocabulary"
"word 'сарбаш' not in vocabulary"
"word 'редкиной' not in vocabulary"
"word 'электромонтаж' not in vocabulary"
"word 'редкиной' not in vocabulary"
"word 'электро+' not in vocabulary"
"word 'редкиной' not in vocabulary"
"word 'редкину' not in vocabulary"
"word 'редкиной' not in vocabulary"
"word 'сарбаш' not in vocabulary"
"word 'туманово' not in vocabulary"
"word 'туманово' not in vocabulary"
"word 'приго

"word 'оспариваемому' not in vocabulary"
"word 'бациев' not in vocabulary"
"word 'поповченко' not in vocabulary"
"word 'шилохвоста' not in vocabulary"
"word 'ксенофонтовой' not in vocabulary"
"word 'шилохвост' not in vocabulary"
"word 'маршалова' not in vocabulary"
"word 'маршалова' not in vocabulary"
"word 'маршалова' not in vocabulary"
"word 'бациева' not in vocabulary"
"word 'поповченко' not in vocabulary"
"word 'дств' not in vocabulary"
"word 'азмере' not in vocabulary"
"word 'бациев' not in vocabulary"
"word 'поповченко' not in vocabulary"
"word 'бациева' not in vocabulary"
"word 'поповченко' not in vocabulary"
"word 'кхп' not in vocabulary"
"word 'кхп' not in vocabulary"
"word 'ыскания' not in vocabulary"
"word 'кхп' not in vocabulary"
"word 'бациев' not in vocabulary"
"word 'поповченко' not in vocabulary"
"word 'весенева' not in vocabulary"
"word 'рмавир' not in vocabulary"
"word 'рмавир' not in vocabulary"
"word 'весенева' not in vocabulary"
"word 'першутова' not in vocabulary"

...

"word 'весеневой' not in vocabulary"
"word 'центрторг' not in vocabulary"
"word 'центрторг' not in vocabulary"
"word 'испрашиваемой' not in vocabulary"
"word 'центрторг' not in vocabulary"
"word 'весенева' not in vocabulary"
"word 'центрторг' not in vocabulary"
"word 'центрторг' not in vocabulary"
"word 'аммофос' not in vocabulary"
"word 'аммофос' not in vocabulary"
"word 'татьи' not in vocabulary"
"word 'ивченкова' not in vocabulary"
"word 'валявина' not in vocabulary"
"word 'валявина' not in vocabulary"
"word 'алтайнефтепродукт' not in vocabulary"
"word 'алтайнефтепродукт' not in vocabulary"
"word 'маковской' not in vocabulary"
"word 'тахтамукайский' not in vocabulary"
"word 'кадастровым' not in vocabulary"
"word 'яблоновском' not in vocabulary"
"word 'кадастровым' not in vocabulary"
"word 'яблоновском' not in vocabulary"
"word 'чеуж' not in vocabulary"
"word 'яблоновском' not in vocabulary"
"word 'тахтамукайский' not in vocabulary"
"word 'тахтамукайский' not in vocabulary"
"word 'пр

"word 'товаросопроводительным' not in vocabulary"
"word 'основани' not in vocabulary"
"word 'марамышкина' not in vocabulary"
"word 'шилохвоста' not in vocabulary"
"word 'ксенофонтовой' not in vocabulary"
"word 'электросетевая' not in vocabulary"
"word 'электросетевая' not in vocabulary"
"word 'шилохвост' not in vocabulary"
"word 'авиаагрегат' not in vocabulary"
"word 'авиаагрегат' not in vocabulary"
"word 'авиаагрегат' not in vocabulary"
"word 'технопром' not in vocabulary"
"word 'технопром' not in vocabulary"
"word 'бестоварной' not in vocabulary"
"word 'шилохвоста' not in vocabulary"
"word 'ксенофонтовой' not in vocabulary"
"word 'электросетевая' not in vocabulary"
"word 'моэск' not in vocabulary"
"word 'эксплуатационному' not in vocabulary"
"word 'бездоговорного' not in vocabulary"
"word 'оборонэнергосбыт' not in vocabulary"
"word 'бездоговорного' not in vocabulary"
"word 'црп' not in vocabulary"
"word 'мрэс' not in vocabulary"
"word 'мрэс' not in vocabulary"
"word 'бездоговорного' 

"word 'першутов' not in vocabulary"
"word 'консалдинг' not in vocabulary"
"word 'першутов' not in vocabulary"
"word 'риника' not in vocabulary"
"word 'риника' not in vocabulary"
"word 'шагиева' not in vocabulary"
"word 'максутовича' not in vocabulary"
"word 'шагиев' not in vocabulary"
"word 'максутович' not in vocabulary"
"word 'шагиева' not in vocabulary"
"word 'бациева' not in vocabulary"
"word 'поповченко' not in vocabulary"
"word 'уруссинская' not in vocabulary"
"word 'бациев' not in vocabulary"
"word 'поповченко' not in vocabulary"
"word 'фбоу' not in vocabulary"
"word 'лубянторг' not in vocabulary"
"word 'кукморский' not in vocabulary"
"word 'лубяны' not in vocabulary"
"word 'кукморский' not in vocabulary"
"word 'лубяны' not in vocabulary"
"word 'лубянторг' not in vocabulary"
"word 'фбоу' not in vocabulary"
"word 'маковской' not in vocabulary"
"word 'фбоу' not in vocabulary"
"word 'фбоу' not in vocabulary"
"word 'дств' not in vocabulary"
"word 'азмере' not in vocabulary"
"word 'ф

"word 'гросула' not in vocabulary"
"word 'водноспортивного' not in vocabulary"
"word 'ксенофонтовой' not in vocabulary"
"word 'шилохвоста' not in vocabulary"
"word 'омскшина' not in vocabulary"
"word 'сибурэнергоменеджмент' not in vocabulary"
"word 'сибурэнергоменеджмент' not in vocabulary"
"word 'балансодержателем' not in vocabulary"
"word 'черемуховская' not in vocabulary"
"word 'черемуховская' not in vocabulary"
"word 'сибурэнергоменеджмент' not in vocabulary"
"word 'шилохвост' not in vocabulary"
"word 'гросула' not in vocabulary"
"word 'незаключенности' not in vocabulary"
"word 'траловая' not in vocabulary"
"word 'траловая' not in vocabulary"
"word 'обжалуемые' not in vocabulary"
"word 'автокомпас' not in vocabulary"
"word 'сибиар' not in vocabulary"
"word 'алтик' not in vocabulary"
"word 'автокомпас' not in vocabulary"
"word 'экспедированию' not in vocabulary"
"word 'весеневой' not in vocabulary"
"word 'дств' not in vocabulary"
"word 'весенева' not in vocabulary"
"word 'весеневой'

"word 'метинвест' not in vocabulary"
"word 'березий' not in vocabulary"
"word 'микаилова' not in vocabulary"
"word 'умуда' not in vocabulary"
"word 'смелковой' not in vocabulary"
"word 'ажнова' not in vocabulary"
"word 'бахруза' not in vocabulary"
"word 'халеддина' not in vocabulary"
"word 'магеррама' not in vocabulary"
"word 'геворгян' not in vocabulary"
"word 'кероба' not in vocabulary"
"word 'аршалуйсовича' not in vocabulary"
"word 'сережаевича' not in vocabulary"
"word 'агаян' not in vocabulary"
"word 'шогик' not in vocabulary"
"word 'микаилова' not in vocabulary"
"word 'умуда' not in vocabulary"
"word 'смелковой' not in vocabulary"
"word 'ажнова' not in vocabulary"
"word 'бахруза' not in vocabulary"
"word 'халеддина' not in vocabulary"
"word 'магеррама' not in vocabulary"
"word 'геворгян' not in vocabulary"
"word 'кероба' not in vocabulary"
"word 'аршалуйсовича' not in vocabulary"
"word 'сережаевича' not in vocabulary"
"word 'агаян' not in vocabulary"
"word 'шогик' not in vocabula

## Doc2Vec
### Задание 3
Напишите функцию обучения doc2vec на юридических текстах, и получите свою кастомную d2v модель. 
> Совет: есть мнение, что для обучения doc2vec модели не нужно удалять стоп-слова из корпуса. Они являются важными семантическими элементами.      

Важно! В качестве документа для doc2vec берите **параграфы** исходного текста, а не весь текст целиком. И не забывайте про предобработку.

In [68]:
texts = []
for idx in tqdm(range(1, 2001)):
    path_file=path+"\\"+str(idx)+'.txt'
    if os.path.isfile(path_file):
        f = open(path_file, 'r', encoding = 'UTF-8')
        file = f.read()
        file = preprocessing(file.split(' '))
        file = ' '.join(file)
        texts.append(file)

A Jupyter Widget

In [70]:
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(texts)]

In [77]:
model_d = Doc2Vec(corpus_file='C:\\Users\\1\\Downloads\\Telegram Desktop\\articles\\article\\texts.txt', vector_size=5, window=2, min_count=1, workers=4)

In [80]:
vector = model_d.infer_vector(["ходатайство"])
vector

array([-0.05459704,  0.0999657 , -0.04365962, -0.0503432 ,  0.09840552], dtype=float32)

### Задание 4
Напишите функцию индексации для поиска через doc2vec. Она должна для каждого документа из корпуса получать вектор.    
Все вектора надо сохранить, по формату советую json. При сохранении не забывайте, что вам надо сохранить не только вектор, но и опознователь текста, которому он принадлежит. 

In [103]:
def make_pars(path):
    file = open(path, 'r', encoding = 'UTF-8')
    f = file.read()
    file.close()
    f = f+'. '
    f = f.lower()
    pars = re.findall('.*?[^А-Я|г]\. ', f)
    pars = preprocessing(pars) 
    return pars

In [106]:
path = r"C:\Users\1\Downloads\Telegram Desktop\articles\article\1.txt"
pars = make_pars(path)

In [102]:
def doc_t_v(par):
    vec = np.zeros(100)
    try:
        vec = model_d.infer_vector(par)
    except Exception as ex:
        print(ex)
    return vec

In [108]:
def get_d2v_vectors(path):
    pars = make_pars(path)
    pars_and_vecs = []
    for i in range(0, len(pars)):
        vec_par = doc_t_v(pars[i])
        pars_and_vecs.append(vec_par.tolist())
    return pars_and_vecs

In [114]:
def write_json_d(pars_and_vecs, global_num):
    p = r'C:\Users\1\Downloads\Telegram Desktop\articles\json\all_json_d2v_' + str(global_num) +'.txt'
    with open(p, 'w') as f:
        json.dump(pars_and_vecs, f)
    return

In [115]:
global_dict = {}
global_num = 0
path = r"C:\Users\1\Downloads\Telegram Desktop\articles\article"
for idx in tqdm(range(1, 2001)):
    path_file=path+"\\"+str(idx)+'.txt'
    if os.path.isfile(path_file):
        global_dict[str(idx)] = get_d2v_vectors(path_file)
        
    if idx%100 is 0:
        global_num+=1
        write_json_d(global_dict, global_num)
        global_dict = {} 

A Jupyter Widget

# Функция поиска

Для обратного индекса функцией поиска является Okapi BM25. Она у вас уже должна быть реализована.

Функция измерения близости между векторами нам пригодится:

In [67]:
from gensim import matutils
import numpy as np 

def similarity(v1, v2):
    v1_norm = matutils.unitvec(np.array(v1))
    v2_norm = matutils.unitvec(np.array(v2))
    return np.dot(v1_norm, v2_norm)

### Задание 5
Напишите функцию для поиска через word2vec и для поиска через doc2vec, которая по входящему запросу выдает отсортированную выдачу документов.

In [58]:
def search_w2v(query):
    query = preprocessing(query.lower().split())
    query_vec = np.zeros(100)
    for word in query:
        vec = model.get_vector(word)
        query_vec += vec
    query_vec = query_vec / len(query)
    path = r"C:\Users\1\Downloads\Telegram Desktop\articles\json\all_json_w2v_"
    doc2similar = {}
    for idx in tqdm(range(1, 21)):
        with open(path + str(idx) + ".txt") as f:
            doc = json.load(f)
        for key, value in doc.items():
            doc2similar[key] = max(model.cosine_similarities(query_vec, [np.array(item) for item in value]))
    

    sorted_dict = sorted(doc2similar.items(), key=operator.itemgetter(1), reverse=True)
    num = sorted_dict[0][0]
    path = r"C:\Users\1\Downloads\Telegram Desktop\articles\article\\"
    with open(path + str(num) + ".txt", encoding = "UTF-8") as f:
        text = f.read()
    return text

In [59]:
query = "В  передаче дела по иску в части взыскания расходов, связанных с предоставлением льгот по оплате услуг отдельным категориям граждан, для пересмотра в порядке надзора судебных актов отказано, так как, удовлетворяя иск, суд исходил из того, что меры социальной поддержки, предусмотренные федеральными законами о социальной защите, являются расходными обязательствами Российской Федерации"
text = search_w2v(query)

A Jupyter Widget

C:\Users\1\Anaconda3\lib\site-packages\gensim\models\keyedvectors.py:920: RuntimeWarning: invalid value encountered in true_divide
  similarities = dot_products / (norm * all_norms)


In [60]:
text

'\n\nОпределение ВАС РФ от 22.12.2011 N ВАС-16916/11 по делу N А76-6807/2011  В  передаче дела по иску в части взыскания расходов, связанных с предоставлением льгот по оплате жилищно-коммунальных услуг отдельным категориям граждан, для пересмотра в порядке надзора судебных актов отказано, так как, удовлетворяя иск, суд исходил из того, что меры социальной поддержки, предусмотренные федеральными законами о социальной защите, являются расходными обязательствами Российской Федерации.'

In [116]:
def search_d2v(query):
    query = preprocessing(query.lower().split())
    query = ' '.join(query)
    query_vec = np.zeros(100)
    query_vec = model_d.infer_vector(query)

    path = r"C:\Users\1\Downloads\Telegram Desktop\articles\json\all_json_d2v_"
    doc2similar = {}
    for idx in tqdm(range(1, 21)):
        with open(path + str(idx) + ".txt") as f:
            doc = json.load(f)
        for key, value in doc.items():
            doc2similar[key] = max(model.cosine_similarities(query_vec, [np.array(item) for item in value]))
    

    sorted_dict = sorted(doc2similar.items(), key=operator.itemgetter(1), reverse=True)
    num = sorted_dict[0][0]
    path = r"C:\Users\1\Downloads\Telegram Desktop\articles\article\\"
    with open(path + str(num) + ".txt", encoding = "UTF-8") as f:
        text = f.read()
    return text

In [117]:
query = "В  передаче дела по иску в части взыскания расходов, связанных с предоставлением льгот по оплате услуг отдельным категориям граждан, для пересмотра в порядке надзора судебных актов отказано, так как, удовлетворяя иск, суд исходил из того, что меры социальной поддержки, предусмотренные федеральными законами о социальной защите, являются расходными обязательствами Российской Федерации"
text = search_d2v(query)

A Jupyter Widget

In [118]:
text

'\n\nКоллегия судей Высшего Арбитражного Суда Российской Федерации в составе председательствующего судьи Харчиковой Н.П., судей Бондаренко С.П.,  Марамышкиной  С.П. рассмотрела в судебном заседании заявление закрытого акционерного общества "Банк ВТБ 24" (Москва) от 16.11.2011 N ИСД-13598 о пересмотре в порядке надзора решения Арбитражного суда Брянской области от 15.03.2011 по делу N А09-447/2010, постановления Двадцатого арбитражного апелляционного суда от  07.06.2011 и постановления Федерального арбитражного суда Центрального округа от 05.10.2011 по тому же делу по иску закрытого акционерного общества "Банк ВТБ 24" в лице операционного офиса "Смоленский" филиала N 3652 ВТБ 24 (г. Смоленск, далее - банк) к Брянскому областному фонду жилищного строительства и ипотеки (г. Брянск, далее - фонд), открытому акционерному обществу "Ипотечная корпорация Брянской области" (г. Брянск, далее - корпорация) о солидарном взыскании задолженности по кредитному договору от 19.12.2006 N 3КЛ-51/2006, в 